# Calculate Z-scores for players

This takes the list of players that we're trying to calculate scores for, and creates z-scores for players with an arbitrarily chosen minimum innings (for pitchers) or plate appearances (for hitters).

Notes for this year's sheet:

- Z-score avg/stdev calculated using minimum IP or PA stats for batting and pitching.
- Reliever value was artificially decremented as usual since they're normally over-inflated value-wise. Multiplied by (project innings) / (90th pctile of projected innings), capped at 1.0.

In [1]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
import psycopg2
import seaborn as sb
import requests

# arbitrarily high max view columns
pd.options.display.max_columns = 150
pd.options.display.max_rows = 200

/Users/zach/Library/Python/3.7/lib/python/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Postgres Connection Information

These are used to get and return a connection to the postgres DB so that we can query for player stats and write them back to the table.

In [2]:
# connection information for the database
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_IP = "192.168.0.118"
POSTGRES_PORT = 5432
POSTGRES_DB = "postgres"

def get_sqlalchemy_engine():
    """
    Create and return a SQLAlchemy engine for inserting into postgres.
    """
    # ## Write Information Back to Database
    #
    return sqlalchemy.create_engine(
        "postgres://{user}:{password}@{host}:{port}/{db}".format(
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD,
            host=POSTGRES_IP,
            port=POSTGRES_PORT,
            db=POSTGRES_DB,
        )
    )

In [3]:
# create a connection and read in pitchers data
engine = get_sqlalchemy_engine()
conn = engine.connect()

## Get Player Data and Create Calculated Columns

For each of the players, we calculate a z-score based on how they compare to the rest of the league. There are several columns that we want to do this for.

- For each z-score, there is an arbitrary minimum set to inflate the league average away from non-full-time players.
- After each is totaled, the z-scores are totaled and then z-scored again to make a single number. This is done so that we can compare pitchers and batters into the same chart.

In [96]:
# read in information from postgres
dfp = pd.read_sql("select * from fantasy.pitchers_projections_depth_charts where ip > 0.0", conn)
dfb = pd.read_sql("select * from fantasy.batters_projections_depth_charts", conn)
dfe = pd.read_sql(sqlalchemy.text(
    """
        select
            fullname as "name",
            eligibility as "elig",
            position as "pos",
            case when position like '%P%' then 'p' else 'b' end as catg,
            injurystatus as "injured",
            averagedraftposition as "espnAdp",
            averagedraftpositionpctchg as "espnAdpChng",
            percentowned as "pctOwn",
            percentstarted as "pctStart",
            auctionValue as "espnAuctionValue"
        from fantasy.players
    """),
    conn
)
dfmcs = pd.read_sql(sqlalchemy.text(
    """
        select
            name,
            exprank as "expRank",
            werth,
            werthrank as "werthRank",
            posadjwerth as "posAdjWerth"
        from fantasy.mrcheatsheet
    """),
    conn
)

In [97]:
# create any calculated columns necessary
dfb['k_pct'] = (dfb['so'] / dfb['ab']).round(2)
dfb['rc'] = (dfb['ab'] * dfb['obp'] * dfb['slg']).round(2)
dfb['catg'] = 'b'

# quality starts metric from here:
# https://yourfantasyjoe.blogspot.com/2010/02/projecting-quality-starts.html
# xQS = GS * (.4650115 - (ERA * .0872381) + ((IP/GS) * .0746775))
dfp['qs'] = (dfp['gs'] * (0.4650115 - (dfp['era'] * 0.0872381) + ((dfp['ip']/dfp['gs']) * .0746775))).round(2)
dfp.fillna(value={'qs': 0.0}, inplace=True)
dfp['catg'] = 'p'

print() # prevent df from printing

### Replace Player Names with Canonical ESPN Names

Multiple inputs have different spellings or variations on player names. The ESPN ones are considered gospel for this analysis.

In [98]:
# source name -> espn name
name_replacements = {
    "Alexander Kirilloff": "Alex Kirilloff",
    "Alexander Lange": "Alex Lange",
    "Alexander Wells": "Alex Wells",
    "Brent Honeywell": "Brent Honeywell Jr.",
    "Caleb Raleigh": "Cal Raleigh",
    "Chad Tromp": "Chadwick Tromp",
    "Cedric Mullins II": "Cedric Mullins",
    "D.J. Johnson": "DJ Johnson",
    "D.J. Stewart": "DJ Stewart",
    "Dan Vogelbach": "Daniel Vogelbach",
    "Daniel Johnson Jr.": "Daniel Johnson",
    "Daniel Poncedeleon": "Daniel Ponce de Leon",
    "Dee Gordon": "Dee Strange-Gordon",
    "Donnie Walton": "Donovan Walton",
    "Eduardo Rodriguez": "Eduardo Rodríguez",
    "Giovanny Urshela": "Gio Urshela",
    "Gregory Deichmann": "Greg Deichmann",
    "Ha-Seong Kim": "Haseong Kim",
    "Ha-seong Kim": "Haseong Kim",
    "Jacob Brentz": "Jake Brentz",
    "Jacob Lamb": "Jake Lamb",
    "Jason Groome": "Jay Groome",
    "Jasrado Chisholm": "Jazz Chisholm",
    "Javier Guerra": "Javy Guerra",
    "Joshua James": "Josh James",
    "Jonathan Brubaker": "JT Brubaker",
    "Kwang hyun Kim": "Kwang-Hyun Kim",
    "Kwang-hyun Kim": "Kwang-Hyun Kim",
    "Lance McCullers": "Lance McCullers Jr.",
    "Lourdes Gurriel": "Lourdes Gurriel Jr.",
    "Lucas Raley": "Luke Raley",
    "Luke Sims": "Lucas Sims",
    "Matt Boyd": "Matthew Boyd",
    "Matthew Cronin": "Matt Cronin",
    "Matthew Manning": "Matt Manning",
    "Michael Reed": "Mike Reed",
    "Michael Soroka": "Mike Soroka",
    "Michael Taylor": "Michael A. Taylor",
    "Nicholas Castellanos": "Nick Castellanos",
    "Nicholas Gordon": "Nick Gordon",
    "Nicholas Lodolo": "Nick Lodolo",
    "Peter Alonso": "Pete Alonso",
    "Ronald Acuna Jr.": "Ronald Acuna",
    "Samuel Clay": "Sam Clay",
    "Samuel Delaplane": "Sam Delaplane",
    "Samuel Long": "Sam Long",
    "Shed Long": "Shed Long Jr.",
    "Shohei Ohtani": "Shohei Ohtani",
    "Steven Souza": "Steven Souza Jr.",
    "Timothy Cate": "Tim Cate",
    "Travis D'Arnaud": "Travis d'Arnaud",
    "Vincent Velasquez": "Vince Velasquez",
    "Will Smith (RP)": "Will Smith",
    "William Vest": "Will Vest",
    "Yandy Diaz": "Yandy Díaz",
    "Yulieski Gurriel": "Yuli Gurriel",
    "Zacary Lowther": "Zac Lowther",
    "Zach Britton": "Zack Britton",
    "Zachary Short": "Zack Short",
    "Zachery Brown": "Zack Brown",
    "Zachery Pop": "Zach Pop",
}

def replace_names(name):
    """
    If a name is in the name_replacements list, get it. Otherwise return the rows name.
    """
    return name_replacements.get(name, name)

# replace all names in all dataframes
for df in [dfb, dfp, dfe, dfmcs]:
    df['name'] = df['name'].apply(lambda x: replace_names(x))

## Calculate Z-Scores

The "thresholds" you see below are used to calculate the z-scores of any particular player based on players who are likely full time players. This is to avoid the long-tail of replacement-level players skewing the average lower.

`min_pa`-type vectors are created below to use as filters for the individual dataframes for batter/pitcher.

In [99]:
# thresholds for batters and pitchers
MINIMUM_INNINGS_PITCHED = 100
MINIMUM_INNINGS_PITCHED_ACTUALS = 20.0
MINIMUM_PLATE_APPEARANCES = 180
MINIMUM_PLATE_APPEARANCES_ACTUALS = 120

# filter columns
min_pa = dfb['pa'] > MINIMUM_PLATE_APPEARANCES
min_ip = dfp['ip'] > MINIMUM_INNINGS_PITCHED

# dict of columns that we want to calculate z-scores for
# +1 means more is better, -1 means negative is better
dfb_score_cols = {
    "pa": {"dir": 1, "weight": 1.25}, 
    "k_pct": {"dir": -1, "weight": 0.9},
    "hr": {"dir": 1, "weight": 1.015},
    "rc": {"dir": 1, "weight": 1.35},
    "woba": {"dir": 1, "weight": 1.325},
    "slg": {"dir": 1, "weight": 1.0},
    "adp": {"dir": -1, "weight": 0.3}
}
dfp_score_cols = {
    "ip": {"dir": 1, "weight": 1.2625},
    "era": {"dir": -1, "weight": 1.0},
    "hr": {"dir": -1, "weight": 0.95},
    "so": {"dir": 1, "weight": 1.05},
    "whip": {"dir": -1, "weight": 1.3},
    "k-9": {"dir": 1, "weight": 1.3},
    "qs": {"dir": 1, "weight": 1.05},
    "gs": {"dir": 1, "weight": 0.25}
}

dfb_weight = 1.0
dfp_weight = 0.85

# get all of the score columns for maximum z-scoring
dfb_score_col_names = [x + "_score" for x in dfb_score_cols.keys()]
dfp_score_col_names = [x + "_score" for x in dfp_score_cols.keys()]

### Calculate Scores and Totals

Totals are used for quickly ranking players across the different P / B groups.

In [100]:
# only mean and stdev are filtered by minimums, so that way the player's actual
# projections are still counted normally.

# batter scores
for col in dfb_score_cols.keys():
    col_score = col + "_score"
    dfb[col_score] = (
        (dfb[col] - dfb[col][min_pa].mean()) / dfb[col][min_pa].std(ddof=0)
        * dfb_score_cols[col]["dir"]
        * dfb_score_cols[col]["weight"]
    ).round(3)
    
# pitcher scores
for col in dfp_score_cols.keys():
    col_score = col + "_score"
    dfp[col_score] = (
        (dfp[col] - dfp[col][min_ip].mean()) / dfp[col][min_ip].std(ddof=0)
        * dfp_score_cols[col]["dir"]
        * dfp_score_cols[col]["weight"]
    ).round(3)

In [101]:
# sum all of the values into 'total_score'
dfb['total_score'] = dfb[dfb_score_col_names].sum(axis=1)
dfb['total_z_score'] = (
    dfb['total_score'] - dfb['total_score'][min_pa].mean()) / dfb['total_score'][min_pa].std(ddof=0)
dfb['total_z_score_rank'] = dfb['total_z_score'].rank(ascending=False)
dfb['total_z_score'] = dfb.total_z_score.round(3) * dfb_weight

dfp['total_score'] = dfp[dfp_score_col_names].sum(axis=1)
dfp['total_z_score'] = (
    dfp['total_score'] - dfp['total_score'][min_ip].mean()) / dfp['total_score'][min_ip].std(ddof=0)
dfp['total_z_score_rank'] = dfp['total_z_score'].rank(ascending=False)
dfp['total_z_score'] = dfp.total_z_score.round(3) * dfp_weight

# sort by score descending
dfb.sort_values(by='total_z_score_rank', inplace=True)
dfp.sort_values(by='total_z_score_rank', inplace=True)


### Inflate League Mean

For the majority of the z-scores that we're taking, the actual values of them are affected by the long-tail of players who don't get much playing time. To offset this, the mean is artifically placed at halfway through our number of drafted players to account for the fact that will be replacement level for our league.

In [102]:
# decrement scores by the mean of all drafted positions
NUM_TEAMS = 8
NUM_BATTERS = 15
NUM_PITCHERS = 10

MIDDLE_BATTER_INDEX = int((NUM_TEAMS * NUM_BATTERS) / 2)
MIDDLE_PITCHER_INDEX = int((NUM_TEAMS * NUM_PITCHERS) / 2)

middle_batter_score = dfb[dfb['total_z_score_rank'] == MIDDLE_BATTER_INDEX]['total_z_score']
dfb['total_z_score'] = dfb['total_z_score'] - float(middle_batter_score)
dfb.reset_index(drop=True)

middle_pitcher_score = dfp[dfp['total_z_score_rank'] == MIDDLE_PITCHER_INDEX]['total_z_score']
dfp['total_z_score'] = dfp['total_z_score'] - float(middle_pitcher_score)
dfp.reset_index(drop=True)

print() # prevent df from printing

## Write to the Database

Prior to writing the excel files, write back to the database for safekeeping.

In [103]:
dfb.to_sql("batters_scores", conn, schema="fantasy", if_exists="replace")
result = conn.execute("grant select on fantasy.batters_scores to public")

dfp.to_sql("pitchers_scores", conn, schema="fantasy", if_exists="replace")
result = conn.execute("grant select on fantasy.pitchers_scores to public")

## Create Draft Sheet

This sheet includes a number of important pieces of information for drafting specifically, so that players can look up by eligibility as well as important stats.

- Name
- Eligibility
- Positions
- Combined scores

In [109]:
# using pandas instead of SQL
cols = ['name', 'catg', 'total_z_score']
df = pd.concat([
    dfb[cols],
    dfp[cols]
]).sort_values(by='total_z_score', ascending=False)
df.rename(columns = {'total_z_score':'score'}, inplace = True)

# merge in ESPN player data
dfz = pd.merge(df, dfe, left_on=['name', 'catg'], right_on=['name', 'catg'], how='left')
dfd = pd.merge(dfz, dfmcs, on='name', how='left')

# fill in missing type values, need to adjust names later
dfd['elig'] = dfd['elig'].astype('str').replace(to_replace='nan', value='')
dfd['pos'] = dfd['pos'].astype('str').replace(to_replace='nan', value='')

# nix any missing rows at this point
# dfd.dropna(inplace=True)

## Create Positional Scarcity Metrics

Players are grouped into the _most scarce_ position that they're eligible for. Scarcity is defined as number of players above replacement value. This creates a map of all positions and the number of players above replacement, and then uses that to group players to which they should be in.

In [110]:
# create a map of positional scarcity
eligibility = {}
for index, player in dfd.iterrows():
    # add all positions to the dict if not yet
    for pos in player['elig'].split("|"):
        if pos not in eligibility:
            eligibility[pos] = 0

        if player['score'] >= 0.0:
            eligibility[pos] += 1

def get_scarcest_eligibility(x):
    """
    Loops through a player's eligibility and puts them in the category that has the least
    number of players that are above replacement level. Arbitrarily set max value unreachably
    high to start so that any eligibility validates.
    """
    best = ""
    best_num = 1000000000
    for pos in x.split("|"):
        pos_num = eligibility[pos]
        if pos_num < best_num:
            best = pos
            best_num = pos_num
    return best

# get the shallowest position that any player is eligible for, as well
# as their primary position (first in the list, generally)
dfd['shallowestPos'] = dfd['elig'].apply(lambda x: get_scarcest_eligibility(x))
dfd['shallowestPosRank'] = dfd.groupby("shallowestPos")["score"].rank("dense", ascending=False)
dfd['primaryPos'] = dfd['pos'].apply(lambda x: x.split("|")[0])
dfd['primaryPosRank'] = dfd.groupby("primaryPos")["score"].rank("dense", ascending=False)
dfd['catgRank'] = dfd.groupby("catg")["score"].rank("dense", ascending=False)
dfd['rank'] = dfd['score'].rank(ascending=False, method='first', na_option='bottom')

## Style Output

In order to make the outputs more usable, apply a number of color stylings and ranking scale sliders to the actual xlsx file.

In [115]:
dfd = dfd[['rank','name','elig','pos','injured','score','espnAdp','espnAdpChng','espnAuctionValue','pctOwn',
    'pctStart','catg','catgRank','primaryPos','primaryPosRank', 'werth', 'werthRank', 'posAdjWerth']]

## Write Excel files for distribution

In [116]:
with pd.ExcelWriter('fantasy2021.xlsx') as writer:
    dfb.to_excel(writer, sheet_name='batters')
    dfp.to_excel(writer, sheet_name='pitchers')
    dfd.to_excel(writer, sheet_name='draft_sheet')

## Draft Sheet Preview

This is a quick look at how the players are going to appear in the final draft sheet. Useful for comparing overall pitcher v batter weights (i.e. verify that nobody should be ranked higher than trout).

In [117]:
dfd.sort_values(by='score', ascending=False).head(150)

,rank,name,elig,pos,injured,score,espnAdp,espnAdpChng,espnAuctionValue,pctOwn,pctStart,catg,catgRank,shallowestPos,shallowestPosRank,primaryPos,primaryPosRank,werth,werthRank,posAdjWerth
0,1.0,Jacob deGrom,P|SP,SP,ACTIVE,2.46330,6.79,0.15,40.0,99.82,99.23,p,1.0,SP,1.0,SP,1.0,11.3,2.0,13.4
1,2.0,Juan Soto,OF|UTIL,OF,ACTIVE,1.99100,5.63,-0.30,43.0,99.80,99.37,b,1.0,OF,1.0,OF,1.0,12.0,1.0,14.3
2,3.0,Gerrit Cole,P|SP,SP,ACTIVE,1.84620,6.34,-0.13,41.0,99.85,99.40,p,2.0,SP,2.0,SP,2.0,8.0,4.0,10.0
3,4.0,Mike Trout,OF|UTIL,OF,ACTIVE,1.80400,4.20,-0.07,42.0,99.84,99.43,b,2.0,OF,2.0,OF,2.0,10.4,3.0,12.8
4,5.0,Shane Bieber,P|SP,SP,ACTIVE,1.57505,11.49,-0.20,37.0,99.82,99.31,p,3.0,SP,3.0,SP,3.0,7.7,5.0,9.8
5,6.0,Max Scherzer,P|SP,SP,ACTIVE,1.49345,24.90,0.06,25.0,99.75,98.87,p,4.0,SP,4.0,SP,4.0,6.1,9.0,8.2
6,7.0,Yu Darvish,P|SP,SP,ACTIVE,1.42375,20.37,-0.50,28.0,99.79,98.59,p,5.0,SP,5.0,SP,5.0,4.7,15.0,6.7
7,8.0,Ronald Acuna,OF|UTIL,OF,ACTIVE,1.35400,3.91,-0.27,46.0,99.81,99.50,b,3.0,OF,3.0,OF,3.0,7.1,8.0,9.4
8,9.0,Trevor Bauer,P|SP,SP,ACTIVE,1.29540,15.19,-0.50,30.0,99.82,98.52,p,6.0,SP,6.0,SP,6.0,3.5,23.0,5.5
9,10.0,Cody Bellinger,1B|1B/3B|OF|UTIL,1B|OF,ACTIVE,1.26800,12.67,1.74,36.0,99.80,98.73,b,4.0,1B,1.0,1B,1.0,7.4,6.0,10.2


# Visualizations

Here are a number of helpful visualizations to sort out the information from the draft sheet.

### ESPN Average Draft Pos vs. this Draft Sheet Ranking

Here's how players stack up against how they're being drafted on ESPN right now

In [ ]:
sb.lmplot(
    x="espnAdp", y="score", hue="primaryPos", fit_reg=False,
    data=dfd.sort_values(by='score', ascending=False).head(200)
)


### Score vs ESPN Average Draft Pick

Score is correlated to our stats, whereas draft pick is correlated to a 5x5 league. This is useful for getting an idea around where positional scarcity is.

In [ ]:
sb.swarmplot(data=dfd[dfd['rank'] <= 200], x="score", y="primaryPos", orient="h")